Attribute Information:

The data set contains raw process sensor data (i.e. without feature extraction) which are structured as matrices (tab-delimited) with the rows representing the cycles and the columns the data points within a cycle. The sensors involved are:

Sensor		Physical quantity		Unit		Sampling rate
PS1		Pressure			bar		100 Hz
PS2		Pressure			bar		100 Hz
PS3		Pressure			bar		100 Hz
PS4		Pressure			bar		100 Hz
PS5		Pressure			bar		100 Hz
PS6		Pressure			bar		100 Hz
EPS1		Motor power			W		100 Hz
FS1		Volume flow			l/min		10 Hz
FS2		Volume flow			l/min		10 Hz
TS1		Temperature			Â°C		1 Hz
TS2		Temperature			Â°C		1 Hz
TS3		Temperature			Â°C		1 Hz
TS4		Temperature			Â°C		1 Hz
VS1		Vibration			mm/s		1 Hz
CE		Cooling efficiency (virtual)	%		1 Hz
CP		Cooling power (virtual)		kW		1 Hz
SE		Efficiency factor		%		1 Hz



Has Missing Values?

No

In [1]:
import pandas as pd
import numpy as np
from numpy import loadtxt
import matplotlib.pyplot as plt

#PS: Pressure, bar, 100 Hz --> 100 Messungen pro Skeunde
#EPS: Motor power, W, 100 Hz
#FS: Volume flow, l/min, 10 Hz --> 10 Messungen pro Sekunde
#TS: Temperature, Celsius, 1 Hz --> 1 Messung pro Sekunde
#VS: Vibration, mm/s, 1Hz
#CE: Cooling efficiency (virtual), %, 1 Hz
#CP: Cooling power (virtual), kW, 1 Hz
#SE: Efficency factor, %, 1 Hz


input_path = "input_data/"
file_names = [
    "ce", "cp", "eps1", "se", "vs1", 
    "fs1", "fs2", 
    "ps1", "ps2", "ps3", "ps4", "ps5", "ps6",
    "ts1", "ts2", "ts3", "ts4", "target"
]

data = {}

for file_name in file_names:
    data[file_name] = loadtxt(f"{input_path}{file_name}.txt")

In [2]:
data['ce'].mean()

31.29907712773998

In [ ]:
for file_name in file_names:
    print(f"Shape of {file_name}:{data[file_name].shape}")

In [39]:
df_target = pd.DataFrame(data['target'], columns=['Cooler_Condition',
                                         'Valve_Condition',
                                         'Internal_Pump_Leakage',
                                         'Hydraulic_Accumulator',
                                         'Stable_Flag'])
df_target = df_target['Valve_Condition']

#df_target.head()

In [ ]:
df_target.describe()

In [41]:
del data['target']

In [ ]:
fig, axes = plt.subplots(6, 3, figsize=(15, 20))
axes = axes.flatten()  


for i, key in enumerate(data):
    ax = axes[i]
    ax.plot(data[key], color = 'blue', linewidth = 1)
    ax.plot(np.mean(data[key], axis=1), color = 'red') 
    ax.set_title(key) 

plt.tight_layout()
plt.show()

In [ ]:
for key, array in data.items():
    print(f"{key}: Mean = {round((np.mean(array)),4)}, Std = {round((np.std(array)),4)}, Min = {np.min(array)}, Max = {np.max(array)}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

for key in data: 
    # Erstelle ein 2D-Array (zum Beispiel ein einfaches Bild oder eine Matrix)
    image = data[key]

    # Berechne die 2D-Fourier-Transformation
    f_transform = np.fft.fft2(image)

    # Verschiebe den Nullfrequenzanteil in die Mitte des Spektrums
    f_transform_shifted = np.fft.fftshift(f_transform)

    # Berechne das Magnitudespektrum (Betrag)
    magnitude_spectrum = np.abs(f_transform_shifted)

    # Berechne das Phasenspektrum (Phase)
    phase_spectrum = np.angle(f_transform_shifted)

    # Summiere das Magnitudespektrum entlang der Spalten (horizontale Richtung)
    magnitude_spectrum_sum_cols = np.sum(magnitude_spectrum, axis=0)

    # Plot des Magnitudespektrums als Summe entlang der Spalten
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(magnitude_spectrum_sum_cols)
    plt.title(f"{key}: Magnitude Spectrum (Summe entlang der Spalten)")
    plt.xlabel("Spalten-Index")
    plt.ylabel("Summierte Magnitude")

    # Plot des Phasenspektrums
    plt.subplot(1, 2, 2)
    plt.imshow(phase_spectrum, cmap='twilight', aspect='auto')
    plt.title(f"{key}: Phase Spectrum")
    plt.colorbar(label='Phase (radians)')
    plt.tight_layout()
    plt.show()


In [5]:
import glob
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks

# Funktion zur Feature-Extraktion
def extract_features(data):
    features = {}
    features['mean'] = np.mean(data)
    features['median'] = np.median(data)
    features['std'] = np.std(data)
    features['min'] = np.min(data)
    features['max'] = np.max(data)
    features['range'] = np.max(data) - np.min(data)
    features['skew'] = skew(data)
    features['kurtosis'] = kurtosis(data)
    diffs = np.diff(data)
    features['mean_diff'] = np.mean(diffs)
    features['std_diff'] = np.std(diffs)
    #peaks, _ = find_peaks(data)
    #valleys, _ = find_peaks(-np.array(data))
    #features['num_peaks'] = len(peaks)
    #features['num_valleys'] = len(valleys)
    return features

all_features = []
for key in data:
    features = extract_features(data[key])
    all_features.append(features)

# In DataFrame speichern
features_df = pd.DataFrame(all_features)

# Speichern
#features_df.to_csv("extracted_features.csv", index=False)


In [6]:
features_df

,mean,median,std,min,max,range,skew,kurtosis,mean_diff,std_diff
0,31.299077,27.303,11.577906,17.042,48.777,31.735,"[0.46982448803187055, 0.46883928758549304, 0.4...","[-1.5068595466091073, -1.506820254024693, -1.5...",-7.640340e-04,0.149860
1,1.808399,1.738,0.279326,1.016,2.909,1.893,"[0.329255121612887, 0.3220476877692316, 0.3174...","[-0.9107545614806969, -0.8989159230526287, -0....",-5.221569e-05,0.010946
2,2495.509203,2425.600,218.222229,2097.800,2995.200,897.400,"[1.571047168145561, 1.5726201791760868, 1.5719...","[8.40861043562447, 8.419407255629826, 8.413934...",9.223004e-07,2.565125
3,55.287900,67.064,25.642184,0.000,100.600,100.600,"[-10.18725754933782, nan, nan, nan, nan, nan, ...","[119.51542636963595, nan, nan, nan, nan, nan, ...",5.995619e-07,13.038785
4,0.613315,0.599,0.080461,0.483,2.546,2.063,"[1.4822647971557785, 1.0744079281164634, 1.357...","[4.947767199961832, 3.3053588460208125, 5.9780...",-4.850532e-04,0.018549
5,6.198549,7.641,3.213826,0.000,20.479,20.479,"[0.5463805341830197, 0.08772579188174448, 0.67...","[2.7062537853292277, 0.5512621808698, -0.36958...",-1.083668e-03,0.927608
6,9.649453,9.687,0.449489,8.764,10.453,1.689,"[-0.24093505770497822, -0.2379813987971141, -0...","[-1.2725226423322509, -1.2720122280122128, -1....",-2.824057e-07,0.019709
7,160.485315,153.940,16.133330,133.130,191.920,58.790,"[7.226137819970674, 7.229852151741438, 7.21147...","[75.62175083603016, 75.67453270767565, 75.4315...",-2.237712e-07,0.161329
8,109.379906,125.700,48.103172,0.000,167.770,167.770,"[8.418875115504763, 8.396713487080392, 8.36583...","[92.93979604300785, 92.62432886842318, 92.1902...",-1.133976e-07,1.265524
9,1.753227,2.047,0.934707,0.000,18.828,18.828,"[-1.9453229143549808, -1.9234893026787097, -2....","[14.241772791633789, 14.221664766035747, 14.74...",-3.166061e-07,0.104680
